In [4]:
import numpy as np
import xml.etree.ElementTree as ET

In [28]:
def dist_to_px(dist, spacing):
    """distance in um (or rather same unit as the spacing)"""
    dist_px = int(round(dist / spacing))
    return dist_px


def mm2_to_px(mm2, spacing):
    return (mm2 * 1e6) / spacing**2


def px_to_mm(px: int, spacing: float):
    return px * spacing / 1000


def px_to_um2(px, spacing):
    area_um2 = px * (spacing**2)
    return area_um2


def get_mask_area(slide, spacing=32):
    """Get the size of a mask in pixels where the mask is 1."""
    mask = np.load(slide)
    counts = np.unique(mask, return_counts=True)
    down = 6
    area = counts[1][1] * down**2
    return area


def points_from_xml(xml_path):
    tree = ET.parse(xml_path)
    root = tree.getroot()
    return len(root[0][0][0])


def create_til_score(xml_path, bulk_path):
    """slide_level_nms"""

    cell_count = points_from_xml(xml_path)
    print(cell_count)

    """Compute TIL score and write to output"""
    til_area = dist_to_px(4, 0.5) ** 2
    tils_area = cell_count * til_area
    stroma_area = get_mask_area(bulk_path)
    tilscore = int((100 / int(stroma_area)) * int(tils_area))
    tilscore = min(100, tilscore)
    tilscore = max(0, tilscore)
    print(f"tilscore = {tilscore}")

In [37]:
xml_path = "/home/u1910100/GitHub/TIAger-Torch/xml/162S.xml"
bulk_path = "/home/u1910100/GitHub/TIAger-Torch/tf_seg_out/tempout/162S_bulk.npy"
create_til_score(xml_path, bulk_path)

14554
tilscore = 100
